# Module 6 Test: Boto3 and AWS

This test covers the fundamental concepts and practical skills for working with AWS services using the Boto3 library in Python.

## Topics Covered:
- AWS fundamentals and setup
- S3 operations (buckets, objects, upload/download)
- DynamoDB operations
- Lambda and SQS/SNS basics
- Practical patterns (error handling, pagination)

## Instructions:
1. Read each question carefully
2. Write your solution in the provided code cell
3. Since AWS credentials may not be available, many questions use mocking or are conceptual
4. For mock-based questions, use `unittest.mock` or `moto` library patterns
5. Run your code to verify it works (where applicable)

## Grading:
- Questions 1-4: Fundamentals (easier)
- Questions 5-8: Intermediate
- Questions 9-12: Advanced

Good luck!

In [ ]:
# Required imports for this test
import boto3
from botocore.exceptions import ClientError, BotoCoreError
from unittest.mock import Mock, MagicMock, patch
import json
from typing import Dict, List, Optional, Any

---
## Part 1: AWS Fundamentals and Setup
---

### Question 1: Understanding Boto3 Clients vs Resources (Easy)

Boto3 provides two ways to interact with AWS services: **clients** and **resources**.

**Task:** Write a function called `explain_boto3_interfaces()` that returns a dictionary explaining:
1. What a boto3 client is and when to use it
2. What a boto3 resource is and when to use it
3. One advantage of each approach

The function should return a dictionary with keys: `'client'`, `'resource'`, `'client_advantage'`, `'resource_advantage'`

**Example output structure:**
```python
{
    'client': 'description...',
    'resource': 'description...',
    'client_advantage': 'advantage...',
    'resource_advantage': 'advantage...'
}
```

In [ ]:
# Your solution for Question 1


### Question 2: AWS Credential Configuration (Easy)

AWS credentials can be configured in multiple ways with a specific precedence order.

**Task:** Write a function called `get_credential_precedence()` that returns a list of the credential sources boto3 checks, **in order of precedence** (first checked to last checked).

Include at least 5 credential sources.

**Hint:** Think about environment variables, config files, IAM roles, etc.

In [ ]:
# Your solution for Question 2


---
## Part 2: S3 Operations
---

### Question 3: Creating an S3 Bucket (Easy)

**Task:** Write a function called `create_s3_bucket(bucket_name: str, region: str) -> dict` that:
1. Creates an S3 bucket with the given name in the specified region
2. Handles the special case for `us-east-1` region (no LocationConstraint needed)
3. Returns the response from the create_bucket call
4. Includes proper error handling for `ClientError`

**Note:** You can use mocking to test this function. The function itself should contain the real boto3 code.

In [ ]:
# Your solution for Question 3


### Question 4: Upload and Download Operations (Easy)

**Task:** Write two functions:

1. `upload_file_to_s3(file_path: str, bucket: str, object_key: str) -> bool`
   - Uploads a local file to S3
   - Returns True on success, False on failure
   - Handles errors gracefully

2. `download_file_from_s3(bucket: str, object_key: str, local_path: str) -> bool`
   - Downloads an S3 object to a local file
   - Returns True on success, False on failure
   - Handles errors gracefully

In [ ]:
# Your solution for Question 4


### Question 5: Generate Presigned URLs (Intermediate)

Presigned URLs allow temporary access to private S3 objects without requiring AWS credentials.

**Task:** Write a function called `generate_presigned_url(bucket: str, object_key: str, expiration: int = 3600, operation: str = 'get_object') -> Optional[str]` that:
1. Generates a presigned URL for either downloading (`get_object`) or uploading (`put_object`)
2. Uses the specified expiration time in seconds (default 1 hour)
3. Returns the URL string or None if generation fails
4. Handles both `get_object` and `put_object` operations

In [ ]:
# Your solution for Question 5


---
## Part 3: DynamoDB Operations
---

### Question 6: DynamoDB CRUD Operations (Intermediate)

**Task:** Create a class called `DynamoDBManager` that provides basic CRUD operations for a DynamoDB table.

The class should have:
1. `__init__(self, table_name: str)` - Initialize with table name and create DynamoDB resource
2. `put_item(self, item: Dict) -> bool` - Add or update an item
3. `get_item(self, key: Dict) -> Optional[Dict]` - Retrieve an item by its key
4. `delete_item(self, key: Dict) -> bool` - Delete an item by its key
5. `scan_table(self) -> List[Dict]` - Return all items in the table

All methods should include proper error handling.

In [ ]:
# Your solution for Question 6


### Question 7: DynamoDB Query with Filter (Intermediate)

**Task:** Write a function called `query_users_by_status(table_name: str, status: str, min_age: int) -> List[Dict]` that:
1. Queries a DynamoDB table named `Users` with a GSI (Global Secondary Index) on `status`
2. Filters results to only include users with age >= min_age
3. Uses KeyConditionExpression for the status query
4. Uses FilterExpression for the age filter
5. Returns the list of matching items

**Assume the table has:**
- Primary key: `user_id` (String)
- GSI: `status-index` on `status` attribute
- Attributes: `user_id`, `name`, `status`, `age`

In [ ]:
# Your solution for Question 7


---
## Part 4: Lambda and Messaging Services
---

### Question 8: Invoke Lambda Function (Intermediate)

**Task:** Write a function called `invoke_lambda(function_name: str, payload: Dict, invocation_type: str = 'RequestResponse') -> Dict` that:
1. Invokes an AWS Lambda function with the given payload
2. Supports both synchronous (`RequestResponse`) and asynchronous (`Event`) invocation types
3. For synchronous calls, parses and returns the response payload as a dictionary
4. For async calls, returns a dictionary with the status code
5. Handles errors and returns an error dictionary on failure

**Return format:**
- Success (sync): The parsed response payload
- Success (async): `{'status': 'accepted', 'status_code': 202}`
- Error: `{'error': 'error message'}`

In [ ]:
# Your solution for Question 8


### Question 9: SQS Message Queue Operations (Advanced)

**Task:** Create a class called `SQSManager` that handles SQS operations:

1. `__init__(self, queue_url: str)` - Initialize with queue URL
2. `send_message(self, body: str, attributes: Optional[Dict] = None) -> Optional[str]` - Send a message, return message ID
3. `send_batch(self, messages: List[Dict]) -> Dict` - Send up to 10 messages in a batch, return success/failure counts
4. `receive_messages(self, max_count: int = 1, wait_time: int = 0) -> List[Dict]` - Receive messages with long polling support
5. `delete_message(self, receipt_handle: str) -> bool` - Delete a processed message

**Note:** The batch send should handle the 10-message limit and return `{'successful': count, 'failed': count}`

In [ ]:
# Your solution for Question 9


---
## Part 5: Practical Patterns
---

### Question 10: Pagination Handler (Advanced)

Many AWS operations return paginated results. Boto3 provides paginators to handle this.

**Task:** Write a function called `list_all_s3_objects(bucket: str, prefix: str = '') -> List[Dict]` that:
1. Uses a paginator to list ALL objects in an S3 bucket (handling pagination automatically)
2. Filters by the given prefix (if provided)
3. Returns a list of dictionaries with `key`, `size`, and `last_modified` for each object
4. Handles empty buckets gracefully (returns empty list)

**Hint:** Use `s3_client.get_paginator('list_objects_v2')`

In [ ]:
# Your solution for Question 10


### Question 11: Retry Logic with Exponential Backoff (Advanced)

AWS operations can fail due to throttling or transient errors. Implementing proper retry logic is essential.

**Task:** Write a decorator called `aws_retry` that:
1. Retries a function up to `max_retries` times (default 3)
2. Uses exponential backoff: wait time = `base_delay * (2 ** attempt)` seconds
3. Only retries on specific exceptions: `ClientError` with throttling error codes (`Throttling`, `ThrottlingException`, `RequestLimitExceeded`)
4. Re-raises the exception after all retries are exhausted
5. Logs each retry attempt (print is fine for this exercise)

**Usage example:**
```python
@aws_retry(max_retries=3, base_delay=1)
def my_aws_operation():
    # AWS code here
    pass
```

In [ ]:
# Your solution for Question 11


### Question 12: Multi-Service Integration (Advanced)

**Scenario:** You need to build a simple data pipeline that:
1. Receives a file upload notification
2. Processes the file metadata
3. Stores results in DynamoDB
4. Sends a notification when complete

**Task:** Write a class called `S3EventProcessor` with the following methods:

1. `__init__(self, dynamodb_table: str, sns_topic_arn: str)` - Initialize with DynamoDB table name and SNS topic ARN

2. `process_s3_event(self, event: Dict) -> Dict` - Process an S3 event notification:
   - Extract bucket name, object key, and size from the event
   - Store metadata in DynamoDB (object_key as primary key, include bucket, size, processed_at timestamp)
   - Publish a success notification to SNS
   - Return a summary dict with status and processed object info

3. `_store_metadata(self, metadata: Dict) -> bool` - Helper to store in DynamoDB

4. `_send_notification(self, message: str, subject: str) -> bool` - Helper to publish to SNS

**Sample S3 event structure (simplified):**
```python
{
    'Records': [{
        's3': {
            'bucket': {'name': 'my-bucket'},
            'object': {'key': 'path/to/file.txt', 'size': 1024}
        }
    }]
}
```

In [ ]:
# Your solution for Question 12


---
## End of Test

Make sure you have:
- Answered all 12 questions
- Included proper type hints in all function signatures
- Added error handling where appropriate
- Written clear, readable code following PEP 8

Submit your completed notebook for grading.
---